In [3]:
import os
base_path = os.getcwd()
calib_data_name = "calib_a53"

In [4]:
import numpy as np
import cv2
import glob
import os
import sys
import yaml  # Using YAML for saving parameters is common and readable

# --- Configuration ---
# Pattern size (number of inner corners per a row and column)
checkerboard_size = (9, 7)
# Square size in world units (e.g., mm or inches). Use 1.0 if you only need camera matrix/distortion
# and not real-world distances, but saving the actual size is good practice.
square_size = 1.5  # Example: 15mm or 1.5cm per square
# Directory to save annotated images (optional)
annotation_dir = "chessboard_corners"
save_chessboard_corner_ann = False
# File to save calibration results
calibration_output_file = "camera_calibration_results.yaml"

# --- Calibration Setup ---
# Termination criteria for corner refinement
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
# Assuming the checkerboard is on the z=0 plane
objp = np.zeros((checkerboard_size[0] * checkerboard_size[1], 3), np.float32)
objp[:, :2] = (
    np.mgrid[0 : checkerboard_size[0], 0 : checkerboard_size[1]].T.reshape(-1, 2)
    * square_size
)

# Arrays to store object points and image points from all images
objpoints = []  # 3D points in real world space
imgpoints = []  # 2D points in image plane

# Create annotation directory if it doesn't exist
if save_chessboard_corner_ann and not os.path.exists(annotation_dir):
    os.makedirs(annotation_dir)  # Use makedirs for creating intermediate directories

# --- Image Processing ---
image_files = sorted(
    glob.glob(os.path.join(calib_data_name, "*.jpg"))
)  # Use os.path.join and sort

if not image_files:
    print(f"Error: No image files found in {calib_data_name}")
    sys.exit(1)

print(f"Found {len(image_files)} images. Processing...")

processed_count = 0
for fname in image_files:
    img = cv2.imread(fname)
    if img is None:
        print(f"Warning: Could not read image {fname}. Skipping.")
        continue

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(
        gray,
        checkerboard_size,
        cv2.CALIB_CB_ADAPTIVE_THRESH
        + cv2.CALIB_CB_FAST_CHECK
        + cv2.CALIB_CB_NORMALIZE_IMAGE,
    )

    # If corners are found, add object points and image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        # Refine the corner positions
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)
        processed_count += 1

        # Draw and save the corners (optional)
        if save_chessboard_corner_ann:
            img_with_corners = img.copy()  # Draw on a copy
            cv2.drawChessboardCorners(
                img_with_corners, checkerboard_size, corners2, ret
            )
            # Construct save path carefully
            save_fname = os.path.join(
                annotation_dir,
                os.path.basename(fname).replace(".jpg", "_corner_plot.jpg"),
            )
            cv2.imwrite(save_fname, img_with_corners)
            # print(f"Saved annotated image: {save_fname}") # Optional: print saved filenames

    else:
        print(f"Warning: Could not find corners in {fname}. Skipping.")


print(
    f"Successfully found corners in {processed_count} out of {len(image_files)} images."
)

# --- Perform Calibration ---
if processed_count == 0:
    print("Error: No images with detected corners available for calibration.")
    sys.exit(1)

print("Calibrating camera...")
# Use the size of the first valid gray image for the image size
image_size = gray.shape[::-1]  # (width, height)

ret, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(
    objpoints, imgpoints, image_size, None, None
)

# Check if calibration was successful (ret is usually a boolean indicating convergence)
if not ret:
    print("Error: Camera calibration failed.")
    sys.exit(1)

print("Camera calibration successful.")

# --- Evaluate and Print Results ---
# Compute and print the average reprojection error
mean_reprojection_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(
        objpoints[i], rvecs[i], tvecs[i], cameraMatrix, distCoeffs
    )
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
    mean_reprojection_error += error
mean_reprojection_error /= len(objpoints)

print("\n--- Calibration Results ---")
print("Camera Matrix (K):\n", cameraMatrix)
print("\nDistortion Coefficients (k1, k2, p1, p2[, k3[, k4, k5, k6]]):\n", distCoeffs)
print(f"\nAverage Reprojection Error: {mean_reprojection_error:.4f} pixels")
print("\nNote: A reprojection error below 1.0 pixel is generally considered good.")

# Note: rvecs and tvecs are rotation and translation vectors *for each image*.
# These define the pose of the checkerboard relative to the camera for each image.
print("\nRotation Vectors (per image):\n", rvecs)
print("\nTranslation Vectors (per image):\n", tvecs)


# --- Save Calibration Results ---
print(f"\nSaving calibration results to {calibration_output_file}...")

# Convert numpy arrays to lists for YAML compatibility
calibration_data = {
    "camera_matrix": cameraMatrix.tolist(),
    "distortion_coefficients": distCoeffs.tolist(),
    "reprojection_error": mean_reprojection_error,
    "image_width": image_size[0],
    "image_height": image_size[1],
    # rvecs and tvecs can also be saved if needed, but are often not required for just the intrinsics
    "rvecs": [r.tolist() for r in rvecs],
    "tvecs": [t.tolist() for t in tvecs],
}

try:
    with open(calibration_output_file, "w") as outfile:
        yaml.dump(calibration_data, outfile, default_flow_style=False)
    print("Calibration results saved successfully.")
except Exception as e:
    print(f"Error saving calibration results: {e}")

Found 89 images. Processing...
Successfully found corners in 82 out of 89 images.
Calibrating camera...
Camera calibration successful.

--- Calibration Results ---
Camera Matrix (K):
 [[3.29249724e+03 0.00000000e+00 1.74094359e+03]
 [0.00000000e+00 3.31736746e+03 2.29680764e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Distortion Coefficients (k1, k2, p1, p2[, k3[, k4, k5, k6]]):
 [[ 1.83096642e-01 -8.85691301e-01 -1.54944774e-03 -7.46600476e-04
   1.33316353e+00]]

Average Reprojection Error: 0.3159 pixels

Note: A reprojection error below 1.0 pixel is generally considered good.

Rotation Vectors (per image):
 (array([[ 0.17109694],
       [-0.24668612],
       [-1.52064242]]), array([[ 0.19779806],
       [-0.22118503],
       [-1.55771748]]), array([[ 0.14708021],
       [-0.24846113],
       [-1.53903634]]), array([[ 0.17317954],
       [-0.30505978],
       [-1.52885638]]), array([[ 0.19822077],
       [-0.23800552],
       [-1.55794207]]), array([[ 0.14832759],
       [-